# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Загрузка данных

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except FileNotFoundError:
    df = pd.read_csv('####')

In [3]:
print('Строки, Столбцы в таблице', df.shape)
display(df.info())
display(df.describe())
df.head()

Строки, Столбцы в таблице (5000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Обзор данных показал что пропущенных значений нет, всего в данных 5 столбцов, 1 из них целевой. Данные сильно различаются по шкалам, но поскольку в линейной регрессии мы будем использовать метод наименьших квадратов, то шкалирование не потребуется.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

**Ответ:** Нет, не изменится.

**Обоснование:** Поскольку обратимая матрица по определению является линейно независимой то умножение на неё это просто замена основания или, другими словами, замена системы координат. Так как алгоритм линейной регрессии это система алгебраических преобразований ему без разницы в какой системе координат работать и он будет(и должен) выдавать один и тот же результат вне зависимости от системы координат. Докажем это математически.


Задача обучения стоит в том что-бы подобрать минимальное значение ошибки для вектора с весами $w$

$$
w = \arg\min_w MSE(Xw, y)
$$

Перемножим матрицу X на обратимую матрицу $P$ и вектор с весами $w_p$

$$
w_P = \arg\min_w MSE(XPw_p, y)
$$

Сделаем замену вектора $w_p$

$$
w_p = P^{-1}w
$$

После замены произведение $PP^{-1} = I$, где $I$ единичная матрица. 

$$
w_P = \arg\min_w MSE(XPP^{-1}w, y)
$$

Поскольку умножение единичной матрицы на любую другую эквивалентно умножению на 1, мы возвращаемся к первоначальной задаче.

$$
w = \arg\min_w MSE(Xw, y)
$$


Таким образом задачи обучения модели на данных матрицы и матрицы умноженной на обратимую матрицу равны.

$$
\arg\min_w MSE(XPw_p, y) = \arg\min_w MSE(Xw, y)
$$

Что и требовалось доказать.

## Алгоритм преобразования

**Алгоритм**

1. Генерируем квадратную матрицу из случайных чисел размерностью 4x4.
2. Если матрица обратима то переходим к шагу 3, в другом случае возвращаемся к шагу 1.
3. Умножаем нашу матрицу на сгенерированную матрицу.
4. Получаем зашифрованный датасет.

**Обоснование**

Предположим у нас есть матрица $C$ полученная в результате умножения матрицы $A$ на обратимую матрицу $B$
$$AB = C$$
Для того что-бы найти исходную матрицу $A$ надо перемножить матрицу $C$ на матрицу обратную $B$
$$A=CB^{-1}$$
Таким образом не зная матрицу $B$ злоумышленник не сможет расшифровать данные.

## Проверка алгоритма

In [4]:
np.random.seed(500)

def generate_invert_matrix(size_tuple: tuple):
    # Зашифруем матрицу
    matrix_size = size_tuple[1]
    random_matrix = np.random.random((matrix_size, matrix_size))
    # Добавим единичную матрицу для того что-бы детерминанта не была равна 0
    invertible_matrix = random_matrix + 0.1 * np.eye(matrix_size)
    # Подтвердим что матрица обратима(перестраховка)
    if np.linalg.matrix_rank(invertible_matrix) == matrix_size:
        print("Matrix is invertible.")
        return invertible_matrix
    else:
        print("Matrix is not invertible.")
        return False

# Подготовим данные
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)

while True:
    invertible_matrix = generate_invert_matrix(X.shape)
    if invertible_matrix is not False:
        break


X_transformed = df.drop('Страховые выплаты', axis=1)
# Зашифруем данные
X_transformed = X_transformed.dot(invertible_matrix)
y_transformed = df['Страховые выплаты']
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_transformed, y_transformed, test_size=0.2, random_state=500
)

# Натренируем линейную регрессию на оригинальных данных
model_original = LinearRegression()
model_original.fit(X_train, y_train)
Y_pred_original = model_original.predict(X_test)
# 
# Натренируем линейную регрессию на трансформированных данных
model_transformed = LinearRegression()
model_transformed.fit(X_train_t, y_train_t)
Y_pred_transformed = model_transformed.predict(X_test_t)
# 
# Оценим модели
mse_original = mean_squared_error(y_test, Y_pred_original)
r2_original = r2_score(y_test, Y_pred_original)
mse_transformed = mean_squared_error(y_test_t, Y_pred_transformed)
r2_transformed = r2_score(y_test_t, Y_pred_transformed)

print("Metrics for Original Data:")
print("Mean Squared Error (MSE):", mse_original)
print("R-squared (R2):", r2_original)

print("\nMetrics for Transformed Data:")
print("Mean Squared Error (MSE):", mse_transformed)
print("R-squared (R2):", r2_transformed)

Matrix is invertible.
Metrics for Original Data:
Mean Squared Error (MSE): 0.11295344568601182
R-squared (R2): 0.39635501640126425

Metrics for Transformed Data:
Mean Squared Error (MSE): 0.11295344568611543
R-squared (R2): 0.3963550164007106


__Вывод__:
Теоретические рассуждения подтвердились, результат действительно один и тот же. Важный момент в том что трансформировать необходимо только независимые признаки, если трансформировать всё вместе то результат получается разный. Это происходит потому что трансформация целевого признака меняет решаюмую задачу.